In [1]:
#!/usr/bin/env python3
import requests
import datetime as dt
import csv

PROM_URL = "http://172.18.16.151:30728"
NAMESPACE = "uc3"
POD = "uc35-edge-67f77448c6-bswtq"

def query_prometheus_range(query, start, end, step="15s"):
    url = f"{PROM_URL}/api/v1/query_range"
    params = {
        "query": query,
        "start": int(start.timestamp()),
        "end": int(end.timestamp()),
        "step": step,
    }
    resp = requests.get(url, params=params)
    if resp.status_code >= 400:
        print("Prometheus Error:", resp.text)
        resp.raise_for_status()
    return resp.json()

def main():
    # last 1 hour, like the UI screenshot
    end = dt.datetime.utcnow()
    start = end - dt.timedelta(hours=5)

    promql = (
        'sum(rate(container_cpu_usage_seconds_total{'
        f'namespace="{NAMESPACE}", pod="{POD}", container!=""'
        '}[1m]))'
    )

    data = query_prometheus_range(promql, start, end, step="15s")

    results = data["data"]["result"]
    if not results:
        print("No data returned")
        return

    samples = results[0]["values"]

    with open("cpu_last_5_hours_highres.csv", "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["timestamp", "datetime_utc", "cpu_cores"])
        for ts, val in samples:
            dt_str = dt.datetime.utcfromtimestamp(float(ts)).isoformat()
            w.writerow([ts, dt_str, val])

    print("CSV saved: cpu_last_hour_highres.csv")
    print(f"Samples: {len(samples)}")

if __name__ == "__main__":
    main()


/tmp/ipykernel_125724/3187358055.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = dt.datetime.utcnow()


CSV saved: cpu_last_hour_highres.csv
Samples: 143


/tmp/ipykernel_125724/3187358055.py:48: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt_str = dt.datetime.utcfromtimestamp(float(ts)).isoformat()
